# Advent of Code 2018
*Phong Nguyen, December 2018*

## Lessons Learnt


### Some utility functions

In [1]:
import math
import re
from collections import deque, defaultdict, Counter
from itertools import chain, combinations
from heapq import heappop, heappush

flatten = chain.from_iterable

from numba import jit

def Input(day):
    "Return input file."
    return open('input{}.txt'.format(day))

def InputString(day):
    "Return the content of the input file as a string."
    return Input(day).read()

def InputRows(day):
    "Return the content of the input file as a list of string, each for a row."
    return InputString(day).splitlines()

def InputInts(day):
    "Return the content of the input file as a list of integers, each for a row."
    return [int(x) for x in InputString(day).splitlines()]

def ints(start, end, step=1): return range(start, end+1, step)

# 2D points
UP, LEFT, DOWN, RIGHT = (0, -1), (-1, 0), (0, 1), (1, 0)

def add_tuples(t1, t2):
     return tuple(sum(x) for x in zip(t1, t2))
        
def Mht_distance(p):
    return abs(p[0]) + abs(p[1])

def argmax(a):
    return a.index(max(a))

def neighbors4(point): 
    "The four neighbors (without diagonals)."
    x, y = point
    return ((x+1, y), (x-1, y), (x, y+1), (x, y-1))

def neighbors8(point): 
    "The eight neighbors (with diagonals)."
    x, y = point 
    return ((x+1, y), (x-1, y), (x, y+1), (x, y-1),
            (x+1, y+1), (x-1, y-1), (x+1, y-1), (x-1, y+1))

def breadth_first(start, goal, moves_func):
    "Find a shortest sequence of states from start to the goal."
    frontier = deque([start]) # A queue of states
    previous = {start: None}  # start has no previous state; other states will
    while frontier:
        s = frontier.popleft()
        if s == goal:
            return path(previous, s)
        for s2 in moves_func(s):
            if s2 not in previous:
                frontier.append(s2)
                previous[s2] = s
                
def path(previous, s): 
    "Return a list of states that lead to state s, according to the previous dict."
    return [] if (s is None) else path(previous, previous[s]) + [s]

def astar_search(start, h_func, moves_func):
    "Find a shortest sequence of states from start to a goal state (a state s with h_func(s) == 0)."
    frontier  = [(h_func(start), start)] # A priority queue, ordered by path length, f = g + h
    previous  = {start: None}  # start state has no previous state; other states will
    path_cost = {start: 0}     # The cost of the best path to a state.

    while frontier:
        (f, s) = heappop(frontier)
        if h_func(s) == 0:
            return path(previous, s)
        for s2 in moves_func(s):
            new_cost = path_cost[s] + 1
            if s2 not in path_cost or new_cost < path_cost[s2]:
                heappush(frontier, (new_cost + h_func(s2), s2))
                path_cost[s2] = new_cost
                previous[s2] = s
                
    return dict(fail=True, front=len(frontier), prev=len(previous))

## Day 1: Chronal Calibration
[Problem Description](https://adventofcode.com/2018/day/1)

In [2]:
def day1a(nums):
    return sum(nums)

nums = InputInts(1)
day1a(nums)

505

In [3]:
def day1b(nums):
    seen = set()
    f = 0
    while True:
        for x in nums:
            f += x
            if f in seen:
                return f
            else:
                seen.add(f)
        
day1b(nums)

72330

In [4]:
def has_two(text):
    return 2 in Counter(text).values()

def has_three(text):
    return 3 in Counter(text).values()
    
def day2a(lines):
    two = sum(1 for l in lines if has_two(l))
    three = sum(1 for l in lines if has_three(l))
    return two * three

lines = InputRows(2)
day2a(lines)

8892

In [5]:
def are_correct(t1, t2):
    diff_count = 0
    for i in range(len(t1)):
        if t1[i] != t2[i]:
            diff_count += 1
        if diff_count > 1:
            return False
    return diff_count == 1
    
def day2b(lines):
    for c in combinations(lines, 2):
        if are_correct(*c):
            t1, t2 = c
            common_string = ''
            for i in range(len(t1)): 
                if t1[i] == t2[i]: 
                    common_string += t1[i]
            print(common_string)
                
%time day2b(lines)

zihwtxagifpbsnwleydukjmqv
CPU times: user 38.1 ms, sys: 1.46 ms, total: 39.5 ms
Wall time: 38.5 ms


A shorter way (less code) and also easier to find the common string for the answer is to remove each character and check if the rest of the two strings are the same. It is slower though.

In [6]:
%%time

for c in combinations(lines, 2):
    t1, t2 = c
    for i in range(len(t1)):
        x1 = t1[:i] + t1[i+1:]
        x2 = t2[:i] + t2[i+1:]
        if x1 == x2:
            print(x1)
            break

zihwtxagifpbsnwleydukjmqv
CPU times: user 585 ms, sys: 6 ms, total: 591 ms
Wall time: 586 ms


Reading solutions in reddit, I see using `zip` is a neat way of looping through pairs of things. Also, a more pythonic way of summing booleans. So, rewrite my first solution.

In [9]:
def are_correct_zip(t1, t2):
    return sum(c1 != c2 for c1, c2 in zip(t1, t2)) == 1
    
def day2b_zip(lines):
    for c in combinations(lines, 2):
        if are_correct(*c):
            common_string = ''.join(c1 for c1, c2 in zip(*c) if c1 == c2)
            print(common_string)
                
%time day2b_zip(lines)

zihwtxagifpbsnwleydukjmqv
CPU times: user 37.8 ms, sys: 1.62 ms, total: 39.5 ms
Wall time: 38 ms


Well, it is as fast as the original one, even though it needs to sum up all differences, without early exit.